In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
from scipy.stats import ks_2samp


In [4]:
#Cargar el modelo y el preprocesador
modelo = joblib.load("model_heuristic.pkl")
preprocesador = joblib.load("preprocessor.pkl")

print("Modelo y preprocesador cargados correctamente.")


Modelo y preprocesador cargados correctamente.


In [13]:
data = pd.read_csv("telecom_churn.csv")

# Variable objetivo CHURN
X_original = data.drop("Churn", axis=1)
y_original = data["Churn"]

print("Dataset original cargado con", X_original.shape[0], "registros.")

Dataset original cargado con 3333 registros.


In [16]:
# Simulación con nuevos datos
X_nuevo = X_original.sample(300, random_state=42).copy()

# Simular drift alterando algunos valores
X_nuevo["AccountWeeks"] = X_nuevo["AccountWeeks"] * np.random.uniform(0.9, 1.1, size=len(X_nuevo))
X_nuevo["DayMins"] = X_nuevo["DayMins"] * np.random.uniform(0.85, 1.15, size=len(X_nuevo))

print("Datos nuevos simulados correctamente.")

Datos nuevos simulados correctamente.


In [19]:
#Comparacion de las estadisticas entre el original y el nuevo
def comparar_distribuciones(df_old, df_new):
    print("\nComparación de distribuciones (prueba KS):")
    resultados = []

    for col in df_old.columns:
        if np.issubdtype(df_old[col].dtype, np.number):
            ks_stat, p_value = ks_2samp(df_old[col], df_new[col])
            drift = "Posible drift" if p_value < 0.05 else "Estable"
            resultados.append([col, round(ks_stat, 4), round(p_value, 4), drift])

    return pd.DataFrame(resultados, columns=["Variable", "KS-Stat", "p-value", "Conclusión"])

tabla_drift = comparar_distribuciones(X_original, X_nuevo)
tabla_drift


Comparación de distribuciones (prueba KS):


,Variable,KS-Stat,p-value,Conclusión
0,AccountWeeks,0.0513,0.4490,Estable
1,ContractRenewal,0.0169,1.0000,Estable
2,DataPlan,0.0034,1.0000,Estable
3,DataUsage,0.0340,0.8963,Estable
4,CustServCalls,0.0301,0.9571,Estable
5,DayMins,0.0435,0.6566,Estable
6,DayCalls,0.0243,0.9955,Estable
7,MonthlyCharge,0.0460,0.5875,Estable
8,OverageFee,0.0396,0.7636,Estable
9,RoamMins,0.0397,0.7607,Estable


In [ ]:
_dntk.DeepnoteChart(tabla_drift, """{"mark":{"type":"bar","tooltip":true},"width":"container","config":{"customFormatTypes":true},"height":"container","$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{"x":{"sort":null,"type":"nominal","scale":{"type":"linear"},"format":{"type":"default","decimals":null}},"y":{"sort":null,"type":"nominal","scale":{"type":"linear"},"format":{"type":"default","decimals":null}}},"usermeta":{"tooltipDefaultMode":true}}""", attach_selection=True, filters='[]')

In [ ]:
_dntk.DeepnoteChart(tabla_drift, """{"mark":{"type":"bar","tooltip":true},"width":"container","config":{"customFormatTypes":true},"height":"container","$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{"x":{"sort":null,"type":"nominal","scale":{"type":"linear"},"format":{"type":"default","decimals":null}},"y":{"sort":null,"type":"nominal","scale":{"type":"linear"},"format":{"type":"default","decimals":null}}},"usermeta":{"tooltipDefaultMode":true}}""", attach_selection=True, filters='[]')

Insights:

-Todas las variables que se analizaron presentan una estabilidad estadística buena, ya que los valores de p-value son mayores a 0.05.

-No hubieron cambios significativos a la hora de distribuir los datos con respecto a los del entrenamiento original.

-Algunas variables como Accountweeks, MonthlyCharge, RoaMins y DayMins se mantienen con comportamientos iguales, lo cual indica que el entorno de negocio no se ha cambiado de una manera abrupta.

-No se detectaron desviaciones significativas, por lo que este modelo conserva su precisión. 

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=4f703592-4945-45ea-93be-011448982d02' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>